In [8]:
'''
    *****Auto Model building using TPOT ***** 

This noteboook is having sample code of TPOTRegressor to generate optimal model file for HousePricing use case.

Note:  
a. Please make sure you have skelarn >= 0.21.0 [python.exe -m pip install --user --upgrade scikit-learn==0.21.0]
b. You need to update Jupyter and ipywidgets libraries to make TPOT work. 
    python.exe -m pip install --user --upgrade jupyter
    python.exe -m pip install --user --upgrade ipywidgets
    
'''

'\n    *****Auto Model building using TPOT ***** \n\nThis noteboook is having sample code of TPOTRegressor to generate optimal model file for HousePricing use case.\n\nNote:  \na. Please make sure you have skelarn >= 0.21.0 [python.exe -m pip install --user --upgrade scikit-learn==0.21.0]\nb. You need to update Jupyter and ipywidgets libraries to make TPOT work. \n    python.exe -m pip install --user --upgrade jupyter\n    python.exe -m pip install --user --upgrade ipywidgets\n    \n'

In [10]:
%matplotlib inline
#!pip install category_encoders
#!pip install tpot
#!pip install sklearn_pandas 
#!pip install xgboost

  Using cached https://files.pythonhosted.org/packages/8d/2f/fcb6150813b89d628749784370132e431f687ebab5a1063eb298cc941f76/scipy-1.4.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/e9/fc/37c2706fe0d252e89c49f0c94b94b27878f75a372ca7e5e7ea7583f61c79/scikit_learn-0.22-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/8c/c3/d049cf3fb31094ee045ec1ee29fffac218c91e82c8838c49ab4c3e52627b/tqdm-4.41.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/17/c9/ab11855af164d03be0ff4fddd4c46a5bd44799a9ecc1770e01a669c21168/update_checker-0.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/11/5b/0ef013e1b5560bfd639272d9c530531a7e0a99926c50fc242840dad5a8d7/deap-1.3.0-cp36-cp36m-win_amd64.whl
  Found existing installation: scipy 1.2.1
    Uninstalling scipy-1.2.1:
      Successfully uninstalled scipy-1.2.1


tensorflow 1.14.0 requires google-pasta>=0.1.6, which is not installed.
tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 40.4.3 which is incompatible.
boto3 1.9.82 has requirement botocore<1.13.0,>=1.12.82, but you'll have botocore 1.13.26 which is incompatible.
boto3 1.9.82 has requirement s3transfer<0.2.0,>=0.1.10, but you'll have s3transfer 0.2.1 which is incompatible.
Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\618757\\AppData\\Local\\Temp\\2\\pip-uninstall-v24g0uff\\users\\618757\\anaconda3\\envs\\mypika\\lib\\site-packages\\scipy\\cluster\\_hierarchy.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import category_encoders as ce
import timeit
# import category_encoders
import os
from math import sqrt
from sklearn.preprocessing import OneHotEncoder, Imputer, StandardScaler, MinMaxScaler, LabelEncoder, normalize
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder, normalize
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,  cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer
# from sklearn_pandas import CategoricalImputer
from sklearn.impute import SimpleImputer
from tpot import TPOTRegressor

%matplotlib inline

In [24]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 300)
df = pd.read_csv("train.csv")

# break into X and y dataframes
X = df.reindex(columns=[x for x in df.columns.values if x != 'SalePrice'])        # separate out X
y = df.reindex(columns=['SalePrice'])   # separate out y
y = np.ravel(y)                     # flatten the y array

# make list of numeric and string columns
numeric_cols = [] # could still have ordinal data
string_cols = []  # could have ordinal or nominal data

for col in X.columns:
    if (X.dtypes[col] == np.int64 or X.dtypes[col] == np.int32 or X.dtypes[col] == np.float64):
        numeric_cols.append(col)      # True integer or float columns
    
    if (X.dtypes[col] == np.object):  # Nominal and ordinal columns
        string_cols.append(col)

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [25]:
'''
Applying data pre-processing techniques to convert data into proper format

'''

# Applying Label encoder for converting categorical attributes into Numerical values
X_string = X[string_cols]
X_string = X_string.fillna("missing")
X_string = X_string.apply(LabelEncoder().fit_transform) 

# imputing missing values with most freqent values for numeric columns
n_imputer = SimpleImputer(missing_values=np.nan, copy = True, strategy = 'most_frequent') # imputing with most frequent because some of these numeric columns are ordinal
X_numeric = X[numeric_cols]
X_numeric = n_imputer.fit_transform(X_numeric)
X_numeric = pd.DataFrame(X_numeric, columns = numeric_cols)

X = pd.concat([X_numeric, X_string], axis=1, join_axes=[X_numeric.index])
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,856.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,8.0,0.0,2003.0,2.0,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,3,1,2,3,3,0,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,3,3,2,5,1,0,1,4,2,6,5,1,1,4,4,2,3,4,4,8,4
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,1262.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,6.0,1.0,1976.0,2.0,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,3,1,2,3,3,0,2,0,24,1,2,0,2,1,1,8,8,2,3,4,1,2,3,1,0,5,1,0,1,4,3,6,4,1,1,4,4,2,3,4,4,8,4
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,920.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,6.0,1.0,2001.0,2.0,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,3,1,2,0,3,0,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,3,2,2,5,1,0,1,4,2,6,4,1,1,4,4,2,3,4,4,8,4
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,756.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,7.0,1.0,1998.0,3.0,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,3,1,2,0,3,0,0,0,6,2,2,0,5,1,1,13,15,2,3,4,0,3,1,3,0,5,1,2,1,4,2,6,2,5,2,4,4,2,3,4,4,8,0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,1145.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,9.0,1.0,2000.0,3.0,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,3,1,2,0,3,0,2,0,15,2,2,0,5,1,1,12,13,1,2,4,2,2,3,0,2,5,1,0,1,4,2,6,4,1,1,4,4,2,3,4,4,8,4


In [26]:
# convert to numpy array so that if gets XGBoost algorithm doesn't throw ValueError: feature_names mismatch: ...
X = X.values

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 55)

In [7]:
# Make a custom metric function for TPOT
# Root mean squared logarithmic error is how Kaggle scores this task
# Can't use custom scorer with n_jobs > 1.  Known issue.

# def custom_metrics_calculation(y_true, y_pred):
#     return np.sqrt(np.mean((np.log(1 + y_pred) - np.log(1 + y_true))**2))

# Make a custom scorer from the custom metric function
# custom_metrics_score_api = make_scorer(custom_metrics_calculation, greater_is_better=False)


# instantiate tpot 
tpot = TPOTRegressor(verbosity=3,  
                    random_state=55, 
                    #scoring=custom_metrics_score_api,
                    periodic_checkpoint_folder="intermediate_results",
                    n_jobs=-1, 
                    warm_start = True,
                    generations=20, 
                    population_size=80,
                    early_stop=8)
times = []
scores = []
winning_pipes = []

# run 2 iterations
for x in range(1):
    start_time = timeit.default_timer()
    tpot.fit(X_train, y_train)
    elapsed = timeit.default_timer() - start_time
    times.append(elapsed)
    winning_pipes.append(tpot.fitted_pipeline_)
    scores.append(tpot.score(X_test, y_test))
    tpot.export('tpot_regressor.py')

# output results
times = [time/60 for time in times]
print('Times:', times)
print('Scores:', scores)   
print('Winning pipelines:', winning_pipes)

30 operators have been imported by TPOT.
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: count:poisson
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:gamma
[04:10:57] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:tweedie
[04:10:57] C:\Users\Administrator\Desktop\xg

[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: count:poisson
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:gamma
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:tweedie
[04:10:58] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Obje

[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: count:poisson
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:gamma
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:tweedie
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Obje

[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: count:poisson
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:gamma
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:tweedie
[04:11:01] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Obje

Saving periodic pipeline from pareto front to intermediate_results\pipeline_gen_1_idx_0_2019.12.31_04-15-29.py
Saving periodic pipeline from pareto front to intermediate_results\pipeline_gen_1_idx_1_2019.12.31_04-15-29.py
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[04:15:32] C:\Users\Administrator\Desktop\xgboost\src\objective\o

...

_pre_test decorator: _random_mutation_operator: num_test=0 l1 was provided as affinity. Ward can only work with euclidean distances..
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
Skipped pipeline #1182 due to time out. Continuing to the next pipeline.
Generation 14 - Current Pareto front scores:
-1	-666098677.8071234	GradientBoostingRegressor(input_matrix, GradientBoostingRegressor__alpha=0.9, GradientBoostingRegressor__learning_rate=0.1, GradientBoostingRegressor__loss=ls, GradientBoostingRegressor__max_depth=6, GradientBoostingRegressor__max_features=0.9500000000000001, GradientBoostingRegressor__min_samples_leaf=5, GradientBoostingRegressor__min_samples_split=6, GradientBoostingRegressor__n_estimators=100, GradientBoostingRegressor__subsample=0.45)
-2	-607697640.8481566	RandomFo

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by MinMaxScaler..
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:pairwise
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:ndcg
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: rank:map
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: gpu:reg:logistic
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: reg:logistic
[07:50:20] C:\Users\Administrator\Desktop\xgboost\src\objective\objective.cc:21: Objective candidate: count:poisson
[07:50:20] C:\U

Skipped pipeline #1440 due to time out. Continuing to the next pipeline.
Generation 17 - Current Pareto front scores:
-1	-651761933.7642536	GradientBoostingRegressor(input_matrix, GradientBoostingRegressor__alpha=0.95, GradientBoostingRegressor__learning_rate=0.1, GradientBoostingRegressor__loss=huber, GradientBoostingRegressor__max_depth=8, GradientBoostingRegressor__max_features=1.0, GradientBoostingRegressor__min_samples_leaf=5, GradientBoostingRegressor__min_samples_split=10, GradientBoostingRegressor__n_estimators=100, GradientBoostingRegressor__subsample=0.55)
-2	-586534348.1905262	ExtraTreesRegressor(LassoLarsCV(input_matrix, LassoLarsCV__normalize=True), ExtraTreesRegressor__bootstrap=False, ExtraTreesRegressor__max_features=0.6500000000000001, ExtraTreesRegressor__min_samples_leaf=1, ExtraTreesRegressor__min_samples_split=3, ExtraTreesRegressor__n_estimators=100)
-4	-574393962.1720461	RandomForestRegressor(GradientBoostingRegressor(GradientBoostingRegressor(LassoLarsCV(input_m

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
Generation 19 - Current Pareto front scores:
-1	-651761933.7642536	GradientBoostingRegressor(input_matrix, GradientBoostingRegressor__alpha=0.95, GradientBoostingRegressor__learning_rate=0.1, GradientBoostingRegressor__loss=huber, GradientBoostingRegressor__max_depth=8, GradientBoostingRegressor__max_features=1.0, GradientBoostingRegressor__min_samples_leaf=5, GradientBoostingRegressor__min_samples_split=10, GradientBoostingRegressor__n_estimators=100, GradientBoostingRegressor__subsample=0.55)
-2	-586534348.1905262	ExtraTreesRegressor(LassoLarsCV(input_matrix, LassoLarsCV__normalize=True), ExtraTreesRegressor__bootstrap=False, ExtraTreesRegressor__max_features=0.6500000000000001, ExtraTreesRegressor__min_samples_leaf=1, ExtraTreesRegressor__min_samples_split=3, ExtraTreesRegressor__n_estimators=100)
-4	-574393962.1720461	RandomForestRegressor(Grad

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
Skipped pipeline #1675 due to time out. Continuing to the next pipeline.
Generation 20 - Current Pareto front scores:
-1	-626117241.3106346	GradientBoostingRegressor(input_matrix, GradientBoostingRegressor__alpha=0.9, GradientBoostingRegressor__learning_rate=0.1, GradientBoostingRegressor__loss=ls, GradientBoostingRegressor__max_depth=6, GradientBoostingRegressor__max_features=0.2, GradientBoostingRegressor__min_samples_leaf=5, GradientBoostingRegressor__min_samples_split=6, GradientBoostingRegressor__n_estimators=100, GradientBoostingRegressor__subsample=0.45)
-2	-586534348.1905262	ExtraTreesRegressor(LassoLarsCV(input_matrix, LassoLarsCV__normalize=True), ExtraTreesRegressor__bootstrap=False, ExtraTreesRegressor__max_features=0.6500000